In [1]:
#import data manipulation libraries
import numpy as np
import pandas as pd

#import data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#import filter warning libraries
import warnings
warnings.filterwarnings('ignore')

#import logging libraries
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename='model.log',
                    filemode='w')
#importing scikit learn libraries
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix


In [16]:
#importing dataset using pandas function
url='https://raw.githubusercontent.com/preet-pandey/Bank_Classification_Model/refs/heads/main/bank.csv'
df=pd.read_csv(url,sep=';')
df.sample(frac=1)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
2885,26,services,single,secondary,no,50,yes,no,cellular,12,aug,153,1,-1,0,unknown,no
3471,46,blue-collar,married,primary,no,1291,yes,no,cellular,14,may,142,3,259,17,failure,no
1489,27,unknown,single,tertiary,no,6421,no,no,telephone,28,jan,147,4,-1,0,unknown,no
2077,30,management,single,tertiary,no,201,yes,no,cellular,17,mar,325,2,-1,0,unknown,yes
368,60,technician,single,primary,no,11262,no,no,cellular,26,aug,323,1,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,39,technician,married,tertiary,no,794,no,no,telephone,2,feb,182,1,-1,0,unknown,no
1581,29,technician,single,tertiary,no,406,yes,no,cellular,17,jul,119,1,-1,0,unknown,no
967,35,blue-collar,married,secondary,no,6411,yes,yes,cellular,30,jan,144,1,266,2,failure,no
4467,43,unemployed,divorced,secondary,no,1577,yes,no,cellular,19,nov,87,1,-1,0,unknown,no
